# ML Forecast model(s)
We use the Snowflake Forecast Machine Learning (ML) model to predict Electric Production values. We will predict for 14 time units (this case months).

See further details on [Snowflake Time-series Forecasting](https://docs.snowflake.com/en/user-guide/ml-functions/forecasting) and [General Model Development](https://docs.snowflake.com/en/developer-guide/snowflake-ml/modeling).

Below mainly SQL is used for model training and prediction, but python alternatives are also given in cases where it is possible to use. Final data visualization requires python module and Altair was selected from available options, because this module is in-built for Notebook environment while others require installation, see [Snowflake Visualization options](https://docs.snowflake.com/en/user-guide/ui-snowsight-visualizations).

First SQL environment is set by the following parameters:

In [ ]:
-- SQL SETUP ------------------
use role ACCOUNTADMIN;
use warehouse COMPUTE_WH;
use database EPRODUCT;
use schema PUBLIC;

Download the [Electric production data](https://www.kaggle.com/datasets/shenba/time-series-datasets?phase=FinishSSORegistration&returnUrl=/datasets/shenba/time-series-datasets/versions/1?resource=download&SSORegistrationToken=CfDJ8CXYA35d3CRDujxBNSrCTMubnmICTc2qhQ3tdqdUE5czuLbJUVQLuiFMRthJv_U7VUZ3kLsBhInfN4pHHka3EGnuwb2b5giXeU3secCI-3nFEOkc0hcaZMOw2xBZLO2JIagGq96E4SS9q3fAMnyAXTGg9OMcW_UNTJma7LlMjNt4oxNMm6IhgdBrTyS34dEq2SvGluLjhc0FUF1SCMvoIWBjiwbixOMib-OEWT2tOjIGBj1cK2O17XUQGA2bmxOq99D1ZDlBlGrbh7H-pUqRQ8SMMZlvtYooYo9xrQjLASOCKs1Xjs2PHabbi_n37WtAvu8RsaLXzDlUf6z7m8gbvXc3zg&DisplayName=Jozsef%20Orban) from *Kaggle* website (Kaggle, source data or mysource, Electric_Production.csv, 7.32 kB) and uploaded to Snowflake through Data / Add data / *Load files into a stage* option.

CSV data after upload automatically is inserted to a table called 'ELECTRIC_PRODUCTION'. That table is referenced from now on in SQL queries.

Table consist of a column 'IPG2211A2N' including the Production data (name originates from the csv file), and 'Date' as production date.

In [ ]:
-- Inspect the first 10 rows of your training data.
-- This is the data that we will use to create your model.
SELECT * FROM ELECTRIC_PRODUCTION LIMIT 10;

These values in the query table cannot be visualized directly, but we can do it using pandas DataFrames! The first step is to get the (current Notebook) session, extract data from the results in a cell (ran previously), then we can also use Snowpark for our analyses:

In [ ]:
# With session we can use SQL query data immediately (see 2 cells below)
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
-- Let's query data again but now for the whole dataset to acces entire dataset through active session.
SELECT * FROM ELECTRIC_PRODUCTION;

In [ ]:
hist_data = cell7.to_pandas()
hist_data.describe()

.describe() function calculates the most important statistical values of a dataframe. It is better to create a plot to fully understand the data,... using Altair!

There are different types of Altair plots, now showing two cases
- plot chart
- line chart, which is better for this situation.

In [ ]:
#import Altair python module for plotting (in-built in SnowPark notebook)
import altair as alt
alt.Chart(hist_data).mark_circle().encode(
    x= "DATE",
    y = "ACTUAL")

In [ ]:
alt.Chart(hist_data).mark_line().encode(
    x='DATE',
    y='IPG2211A2N'
)

In [ ]:
-- Prepare your training data. Timestamp_ntz is the required format for time values.
-- Also, only include below selected DATE (converted to timestamp) and value ('IPG2211A2N') columns.
    
CREATE OR REPLACE VIEW ELECTRIC_PRODUCTION_v1 AS SELECT
    to_timestamp_ntz(DATE) as DATE_v1,
    IPG2211A2N
FROM ELECTRIC_PRODUCTION;

Check firts and last 10 rows from dataset.

In [ ]:
SELECT * FROM ELECTRIC_PRODUCTION LIMIT 10;

In [ ]:
SELECT * FROM ELECTRIC_PRODUCTION
ORDER BY DATE DESC LIMIT 10;

Checking the whole dataset it can be recognised that data has equidistant time steps (one value per month), which is the easiest situation for a forecast model to analyse. The latest date is 2018-01-01.

From the plot; the data shows a saturation curve-like model with a fluctuation with defined periodicity in time. This is not a simple linear increase, neither a simple saturation curve, so it is an ideal case (dataset) for ML prediction.

Let's find earliest and latest date to know the time range of the dataset along with the amount of rows. (1985-01-01, 2018-01-01, 397)

In [ ]:
SELECT MIN(DATE), MAX(DATE), COUNT(*)
FROM ELECTRIC_PRODUCTION;

## CREATE PREDICTIONS
- Create your model (using SQL to call SNOWFLAKE.ML.FORECAST library).

In [ ]:
CREATE SNOWFLAKE.ML.FORECAST eProduct_forecast_model(
    INPUT_DATA => SYSTEM$REFERENCE('VIEW', 'ELECTRIC_PRODUCTION_v1'),
    TIMESTAMP_COLNAME => 'DATE_v1',
    TARGET_COLNAME => 'IPG2211A2N'
);

In [ ]:
# not required to run only in special cases
from snowflake.ml.registry import Registry
reg = Registry(session=sp_session, database_name="ML", schema_name="REGISTRY")

In [ ]:
-- to verify whether the model has been instanciated (created)
SHOW SNOWFLAKE.ML.FORECAST INSTANCES

In [ ]:
CREATE TABLE my_forecasts_eProduct AS
SELECT * FROM TABLE(EPRODUCT_FORECAST_MODEL!FORECAST(FORECASTING_PERIODS => 14, CONFIG_OBJECT => {'prediction_interval': 0.95}));

In [ ]:
-- run only if above code does not function!
BEGIN
    call eProduct_forecast_model!FORECAST(FORECASTING_PERIODS => 14);
    LET x := SQLID
    CREATE TABLE My_forecasts_2024_11_04 AS SELECT * FROM TABLE(RESULT_SCAN(:x));
END;

In [ ]:
-- View your predictions.
SELECT * FROM my_forecasts_eProduct;

In [ ]:
DESC TABLE my_forecasts_eProduct;

In [ ]:
# Import python packages, pandas for easy data handling
import pandas as pd

In [ ]:
-- Union your predictions with your historical data, then view the results in a chart.
SELECT DATE, IPG2211A2N AS actual, NULL AS forecast, NULL AS lower_bound, NULL AS upper_bound
    FROM ELECTRIC_PRODUCTION
UNION ALL
SELECT ts as DATE, NULL AS actual, forecast, lower_bound, upper_bound
    FROM my_forecasts_eProduct;

In [ ]:
# Easiest way to read data from SQL query results using 
# session data from above. Converting to a pandas dataframe 
# type variable works only if:
# 1) session is loaded (see get_session() command 2 cells above!)
# 2) a table is formed in a previous step (cell) and cellNo is 
# properly defined below:
my_forecast_df = cell26.to_pandas()
my_forecast_df

In [ ]:
#checking the 'DATE' column's data format.
# It should be Datetime or Timestamp for proper time based plotting
type(my_forecast_df['DATE'][0])

In [ ]:
#a way to convert data to Datetime format (if needed)
datetime_df = pd.to_datetime(my_forecast_df['DATE']).apply(lambda x: x.date())
datetime_df

In [ ]:
# subsets of the full dataframe dataset may be selected
plot_df = my_forecast_df[['DATE', 'ACTUAL']]
plot_df

In [ ]:
# chosing the 'ACTUAL' column values and
# only a few from the beginning (head)
my_forecast_df['ACTUAL'].head()

In [ ]:
#import Altair python module for plotting
# (in-built in SnowPark notebook, ...
# other visualization module(s) should be installed first, see docs)
import altair as alt

In [ ]:
alt.Chart(plot_df).mark_circle().encode(
    x= "DATE",
    y = "ACTUAL")

In [ ]:
base = alt.Chart(my_forecast_df).encode(x='DATE')

alt.layer(
    base.mark_line(color='gray').encode(y='ACTUAL'),
    base.mark_line(color='red').encode(y='UPPER_BOUND'),
    base.mark_line(color='blue').encode(y='LOWER_BOUND')
)

In [ ]:
#my_forecast_df[['DATE', 'ACTUAL']]
base = alt.Chart(my_forecast_df.tail(30)).encode(x='DATE').properties(
    title='Electronic Product - 14 days forecast'
)

alt.layer(
    base.mark_line(color='gray').encode(y='ACTUAL'),
    base.mark_line(color='red').encode(y='UPPER_BOUND'),
    base.mark_line(color='blue').encode(y='LOWER_BOUND')
)

As a final step
# INSPECT RESULTS

- Inspect the accuracy metrics of your model.
- Inspect the relative importance of your features, including auto-generated features. 

In [ ]:
-- Inspect the accuracy metrics of your model.
CALL eProduct_forecast_model!SHOW_EVALUATION_METRICS();

In [ ]:
-- Inspect the relative importance of your features, including auto-generated features. 
CALL eProduct_forecast_model!EXPLAIN_FEATURE_IMPORTANCE();

# Test for known values
Let's roll back time and act like we don't know the last 10 values of the dataset. We remove last values to train our ML Forecast model.

In [ ]:
CREATE OR REPLACE VIEW ELECTRIC_PRODUCTION_v2 AS SELECT
    to_timestamp_ntz(DATE) as DATE_v2,
    IPG2211A2N
FROM ELECTRIC_PRODUCTION
WHERE DATE < '2017-04-01';

In [ ]:
SELECT * FROM ELECTRIC_PRODUCTION_v2
ORDER BY date_v2 DESC limit 10;

## Create model & predicitions

In [ ]:
CREATE OR REPLACE SNOWFLAKE.ML.FORECAST eProduct_forecast_model2(
    INPUT_DATA => SYSTEM$REFERENCE('VIEW', 'ELECTRIC_PRODUCTION_v2'),
    TIMESTAMP_COLNAME => 'DATE_v2',
    TARGET_COLNAME => 'IPG2211A2N'
);

In [ ]:
SHOW SNOWFLAKE.ML.FORECAST INSTANCES

session.call is not yet compatible with models created by ML functions. To call such a model in Snowpark
https://docs.snowflake.com/en/guides-overview-ml-functions

In [ ]:
# Run only in case SQL query doesn not work for forecasting!
from snowflake.ml.registry import Registry
reg = Registry(session=sp_session, database_name="ML", schema_name="REGISTRY")
# instead of direct call like:
# call eProduct_forecast_model2!FORECAST(FORECASTING_PERIODS => 14)'
# use this instead:
session.sql('call eProduct_forecast_model2!FORECAST(FORECASTING_PERIODS => 14)').collect();

In [ ]:
CREATE TABLE my_forecasts_eProduct2 AS
SELECT * FROM TABLE(EPRODUCT_FORECAST_MODEL2!FORECAST(FORECASTING_PERIODS => 14, CONFIG_OBJECT => {'prediction_interval': 0.95}));

In [ ]:
-- A SQL function based SOLUTION to forecast. Use only if above cell did not succeed!
BEGIN
    call eProduct_forecast_model2!FORECAST(FORECASTING_PERIODS => 14);
    LET x := SQLID
    CREATE TABLE My_forecasts_2024_11_27 AS SELECT * FROM TABLE(RESULT_SCAN(:x));
END;

In [ ]:
#repeating import just to be sure that it would work in next step 
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
SELECT DATE, IPG2211A2N AS actual, NULL AS forecast, NULL AS lower_bound, NULL AS upper_bound
    FROM ELECTRIC_PRODUCTION
UNION ALL
SELECT ts as DATE, NULL AS actual, forecast, lower_bound, upper_bound
    FROM my_forecasts_eProduct2;

In [ ]:
my_forecast_df2 = cell50.to_pandas()
my_forecast_df2

In [ ]:
my_forecast_df2.tail(19).head(10)

In [ ]:
my_forecast_df2.describe()

Recombine (merge) real hisorical data points cut off previously with the data used for modeling; unite the whole known dataset. First, get the last 12 values to fill in the predicted dataset's 'ACTUAL' column.

In [ ]:
SELECT DATE, IPG2211A2N FROM ELECTRIC_PRODUCTION
ORDER BY DATE DESC
LIMIT 12;

In [ ]:
hist_data_end = cell55.to_pandas()
hist_data_end

In [ ]:
hist_data_end.describe()
#hist_data_end.loc[11,'IPG2211A2N']

In [ ]:
for i in range(len(hist_data_end)):
    my_forecast_df2.loc[[397+i], 'ACTUAL'] = hist_data_end['IPG2211A2N'][11-i]

my_forecast_df2.tail(19).head(10)

Let's check the predicted values and compare with the real values.

In [ ]:
# import ALtair module if have not imported yet
import altair as alt
# Note: none of the officially suggested Y-axis range limitation worked for me, but the plot was created.
# y = alt.Y('IPG2211A2N').scale(domain=(80, 140), clamp=True))\
# y = alt.Y('IPG2211A2N', scale = alt.Scale(domain=[80, 140]))\
base = alt.Chart(my_forecast_df2.tail(25)).\
    encode(x = alt.X('DATE'),\
           y = alt.Y('IPG2211A2N').scale(domain=(80, 140), clamp=True))\
    .properties(title='Electronic Product - 14 days forecast')

alt.layer(
    base.mark_line(color='gray').encode(y='ACTUAL'),
    base.mark_line(color='red').encode(y='UPPER_BOUND'),
    base.mark_line(color='blue').encode(y='LOWER_BOUND'),
    base.mark_line(color='yellow').encode(y='FORECAST')
)

# Conclusion
The model is not perfect because one single historical data falls below the LOWER_BOUND (2018-03-01), but other real values fall in the determined [LOWER_BOUND & UPPER_BOUND] range. On the other hand we can state that forecasted values (in yellow) are close to historical data.

For a better model, more data or more/altered specifications are required for model training. New models with modified settings or withm ore data Then, you just have to use the best model (taking into account and avoiding the option of overfitting!). 

# Saving data to Table
Save the data in the dataframe to a Table in the Snowflake Database ([save_as_table](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/snowpark/api/snowflake.snowpark.DataFrameWriter.save_as_table)).

Note that first pandas dataframe has to be converted to Spark dataframe, because the first one not, but the latter supplies saving to Table. And one more step to do in advance, get active session to create the Spark dataframe using session.create_dataframe() method.

In [ ]:
# df.write.mode("overwrite").save_as_table("my_table", table_type="temporary"
session = get_active_session()
my_forecast_spark_df = session.create_dataframe(data=my_forecast_df2)
session.table('DataForecast').collect()


In [ ]:
# write data to table
my_forecast_spark_df.write.mode('overwrite').save_as_table('DataForecast')

In [ ]:
df = session.sql('SELECT * FROM DATAFORECAST')
df.collect()
df.order_by('date', ascending = False).limit(10)

In [ ]:
# or verify data writing into table, using SQL query with connection
# this requires sqlalchemy to be imported in Snowflake environment!
#df_table_check = read
import pandas as pd # or create Spark dataframe
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
#    schema = 'xxxx',
#     warehouse = 'xxx',
#     authenticator ='https://xxxxx.authenticator.com'
url = URL(
    account = 'xxxx',
    user = 'xxxx',
    password = 'xxxx*',
    database = 'xxxx',
    role='xxxx',
)
engine = create_engine(url)

connection = engine.connect()
df_table_check = pd.read_sql('SELECT * FROM DATAFORECAST', connection)